In [1]:
import itertools
import gensim
from gifter.modeling.data import lemmatized_frame
from gifter.modeling.tokenizer import lemmatize
from gensim import corpora
from nltk.corpus import brown

In [2]:
def LDA(texts=["default"], num=2, passes=100):
    if (texts == ["default"]):
        df = lemmatized_frame("../data.json", with_tags=False)
        texts = [df['lemmas'].irow(i) for i in range(df.shape[0])]
    else:
        texts2 = []
        for d in texts:
            texts2 = texts2 + [lemmatize(d, with_tags=False)]
            texts = texts2
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    #print "corpus: " + str(corpus)
    corpora.MmCorpus.serialize('./corpus.mm', corpus)
    mm_corpus = corpora.MmCorpus('./corpus.mm')
    #print "mm_corpus: " + str(mm_corpus)
    id2word = {}
    for word in dictionary.token2id:
        id2word[dictionary.token2id[word]] = word
    #lda = gensim.models.ldamodel.LdaModel(
    lda = gensim.models.ldamulticore.LdaMulticore(
        corpus=mm_corpus,
        num_topics=num,
        id2word=id2word,
        #update_every=1,
        eval_every=1,
        passes=passes,
        workers=1
    )
    for i in range(0, lda.num_topics):
        print "Topic number " + str(i) + " consists of words : " + lda.print_topic(i)
    return lda, dictionary

Ponizej algorytm nauczony jest na kontach 2 sportowców i 2 polityków na Twiterze

In [3]:
l, dic = LDA()

/home/marta/Desktop/Gifter/gifter/modeling/tokenizer.py:55: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  pattern = re.compile("(#|RT |{})".format(unidecode(to_remove)), re.I)


Topic number 0 consists of words : 0.016*state + 0.010*secretary + 0.009*win + 0.009*curry + 0.008*lead + 0.008*game + 0.008*minister + 0.007*conservative + 0.007*amp + 0.006*steph
Topic number 1 consists of words : 0.029*fcb + 0.013*president + 0.010*atleti + 0.009*obama + 0.009*atm + 0.008*campion + 0.008*title + 0.008*climate + 0.008*best + 0.007*celebrate


In [4]:
def find_topic(new_doc="User", dictionary=dic, lda=l):
    if (new_doc == "User"):
        df = lemmatized_frame("../test.json", with_tags=False)
        new_doc = list(
            itertools.chain(
                *[df['lemmas'].irow(i) for i in range(df.shape[0])]
            )
        )
    else:
        new_doc = new_doc.lower().split()
    new_vec = dictionary.doc2bow(new_doc)
    vec_lda = sorted(lda[new_vec], key=lambda vec: vec[1], reverse=True)
    print str(vec_lda)
    print "Top topic is topic number " + str(
        vec_lda[0][0]
    ) + " consists of words : " + lda.print_topic(vec_lda[0][0])
    return vec_lda


Poniżej sprawdzony został zbiór tweetów dla użytkownika @BBC Politics

In [5]:
%time find_topic()

[(1, 0.64588987562353839), (0, 0.35411012437646155)]
Top topic is topic number 1 consists of words : 0.029*fcb + 0.013*president + 0.010*atleti + 0.009*obama + 0.009*atm + 0.008*campion + 0.008*title + 0.008*climate + 0.008*best + 0.007*celebrate
CPU times: user 881 ms, sys: 52.1 ms, total: 933 ms
Wall time: 921 ms


[(1, 0.64588987562353839), (0, 0.35411012437646155)]

Poniżej został sprawdzony zbiór tweetow dla użytkownika @ESPNBoxing

In [8]:
%time find_topic()

[(1, 0.67294637348706599), (0, 0.32705362651293401)]
Top topic is topic number 1 consists of words : 0.029*fcb + 0.013*president + 0.010*atleti + 0.009*obama + 0.009*atm + 0.008*campion + 0.008*title + 0.008*climate + 0.008*best + 0.007*celebrate
CPU times: user 535 ms, sys: 8.99 ms, total: 544 ms
Wall time: 537 ms


[(1, 0.67294637348706599), (0, 0.32705362651293401)]